# Practical Optimisations for Pandas
## Eyal Trabelsi

- thanks for coming out, I am Eyal I live in Tel Aviv and work for Salesforce.
- and today I am going to talk to you about Practical Optimisations for Pandas.


## What's Pandas?🐼


A quick reminder to what is pandas?

- Library for data manipulation

- Dataset on Memory

- Widely used 

- So the first question i want to tackle is why we should care about performance?

## Why 🤨

- Fast is better than slow 🐇


- Memory effient is good 💾


- Saving money is good [💸](https://aws.amazon.com/ec2/pricing/on-demand/)


- Hardware will only take you so far 💻

![](https://vignette.wikia.nocookie.net/memepediadankmemes/images/8/80/Acb.jpg/revision/latest/scale-to-width-down/340?cb=20180822064733)

- Ok now that i have got you attention, the next question i want to tackle is when should we optimize our code

## When ⏰

- We should first thrive for program readability, as python aim to make the programmer life easier.

- so All optimization are premature.
- unless


- Program doesn't meet requirement 🚔


- wheter its too slow for the user
- or whether its take too much memory

- Program execution effects development pace 👷

- Every step of development 🐾


- But what parts of the code should we optmize?

## What Parts🎃

- [Identify the bottlenecks](https://www.youtube.com/watch?v=DUCMjsrYSrQ)

- We should only optimize the program bottlenecks
- whether memory and execution time in order to be productive
- There is a great talk about how it and the link is provieded

- [Writing Tests](https://www.youtube.com/results?search_query=tdd+is+dead)

- Like every refactoring task, you want to have same behaviour and return the same restult.
- The best way to achieve this is by writting tests that will make us feel safe

![](https://i.redd.it/8ku1ygqwuuj11.jpg)

- Many people tend to think its problem is python  itself and we cant do anything about

![](https://i.chzbgr.com/full/9093474560/hA8EAB03B/panda-darth-panda-finds-your-lack-of-faith-disturbing-wckpemecom)

- But this is actually not the case.

- The one milion dollar question left to answer is.
- how can we optimize our pandas code.
- and here i will delve into details

## How 👀

- Use What You Need 💾⌛

- Dont reinvent the wheel ⌛💾

- Avoid Loops ⌛

- Picking the Right Types 💾

- Pandas Usage ⌛💾

- Compiled Code ⌛

- General Pyhon Optimization ⌛💾

- Others ⌛💾

## Dataset 📉

In [19]:
! pip install numba numexpr

In [ ]:
import math
import time
import warnings
from dateutil.parser import parse

import janitor
import numpy as np
import pandas as pd
from numba import jit
from sklearn import datasets
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [1]:
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
pd.options.display.max_columns = 999

In [2]:
path = 'https://raw.githubusercontent.com/FBosler/you-datascientist/master/invoices.csv'

def load_dataset(naivly=False):
    df = pd.concat([pd.read_csv(path).clean_names().remove_columns(["meal_id", "company_id"]) for i in range(20)])
    df["meal_price"] = df["meal_price"].astype(int)
    df["meal_tip"] = df["meal_price"].map(lambda x: x * 0.2)
    if naivly:
        for col in df.columns:
            df[col] = df[col].astype(object)
    return df

In [3]:
df = load_dataset()
df.head()

,order_id,date,date_of_meal,participants,meal_price,type_of_meal,heroes_adjustment,meal_tip
0,839FKFW2LLX4LMBB,2016-05-27,2016-05-31 07:00:00+02:00,['David Bishop'],469,Breakfast,False,93.8
1,97OX39BGVMHODLJM,2018-09-27,2018-10-01 20:00:00+02:00,['David Bishop'],22,Dinner,False,4.4
2,041ORQM5OIHTIU6L,2014-08-24,2014-08-23 14:00:00+02:00,['Karen Stansell'],314,Lunch,False,62.8
3,YT796QI18WNGZ7ZJ,2014-04-12,2014-04-07 21:00:00+02:00,['Addie Patino'],438,Dinner,False,87.6
4,6YLROQT27B6HRF4E,2015-07-28,2015-07-27 14:00:00+02:00,['Addie Patino' 'Susan Guerrero'],690,Lunch,False,138.0


- Our dataset is a 1m data that contain meal invoices
- Before i begin its important to state that all these optimization depends on the size of your data and for small dataset might be unrelevant
- The first two optimizations i am going to cover are pretty stright forward.


## Use What You Need 🧑

- Load required columns only

- For many use cases you dont actually need the entire dataset
- There might be unneeded columns which we dont use in our analysis or data manipulation and this can save a lot of memory

- Sampling when possible

- For your usecase you might dont need all the rows.
- For example if you want to understand the dataset 1m rows probably will suffice
- which will reduce memory and execution time

## Dont reinvent the wheel 🎡

- Vast ecosystem

- Use existing solutions

- For example instead of implmenting kmeans by your own
- use scipy/sckitlearn implementation, not only it will save you bugs
- but it will probably be highly optimized

## Avoid Loops ♾

- Pandas is design for vector manipulations
- Which make loops inefficient
- a Rookie mistake in pandas will be to just loop over all the rows" either using iterows or regular iterations

### Bad Options 😈

In [4]:
def iterrows_original_meal_price(df):
    for i, row in df.iterrows():
        df.loc[i]["original_meal_price"] = row["meal_price"] - row["meal_tip"]
    return df

In [5]:
%%timeit -r 1 -n 1
iterrows_original_meal_price(df)

35min 13s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


- As you might of guessed both  bring us to unsatisfied results of half an hour.
- there is a much Nicer way is to use map/apply.

### Better Option 🤵

- apply accepts any user defined function that applies a transformation/aggregation on a DataFrame. 
- apply is effectively a silver bullet that does whatever any existing pandas function cannot do.

In [4]:
def apply_original_meal_price(df):
    df["original_meal_price"] = df.apply(lambda x: x['meal_price'] - x['meal_tip'], axis=1)
    return df

In [5]:
%%timeit 
apply_original_meal_price(df)

22.5 s ± 170 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- This starts to look better as we got to 21 seconds and about x100 faster but can we do any better?
- The obvious answer is yes

## 100x Improvement In Execution Time ⌛

### Best Option 👼

In [6]:
def vectorized_original_meal_price(df):
    df["original_meal_price"] = df["meal_price"] - df["meal_tip"] 
    return df

- Vectorization is the process of executing operations on entire arrays.
- Pandas includes a generous collection of vectorized functions for everything from mathematical operations to aggregations.

In [7]:
%%timeit 
vectorized_original_meal_price(df)

5.05 ms ± 374 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


- We can see the benefit of vetorized function right away we got from  to 2ms and about 100,00.


## Another 8000x Improvement In Execution Time ⌛

## Additional Resources 📚

- [Vectorization Mindset](https://www.youtube.com/watch?v=EEUXKG97YRw)

- [Numpy](https://numpy.org/doc/stable/reference/index.html)/[Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)/[Scipy](https://docs.scipy.org/doc/scipy/reference/special.html) vectorizations methods

- Since writing vectorized functions require a bit of a learning curve i have added an awesome talk about how it
- Also there are links for vectorized methods from scipy/numpy and pandas

## Picking the Right Type 🌈

- so lets see where we start, i have loaded the dataframe with very naive types in order to emphasis the change
- but in most use cases you could expect to gain half the memory foot print

In [8]:
df = load_dataset(naivly=True)

### Dataframe Size 🏋️

In [9]:
df.memory_usage(deep=True).sum()

478844140

### Columns Sizes 🏋️

In [10]:
df.memory_usage(deep=True)

Index                 8002720
order_id             73024820
date                 67022780
date_of_meal         82027880
participants         84977580
meal_price           36012240
type_of_meal         63688760
heroes_adjustment    32076480
meal_tip             32010880
dtype: int64

In [11]:
df.dtypes

order_id             object
date                 object
date_of_meal         object
participants         object
meal_price           object
type_of_meal         object
heroes_adjustment    object
meal_tip             object
dtype: object

- I hope its clear that the type of the column effect the memory foot print.
- There is a great picture explain

![](https://hackr.io/blog/uploads/images/1570190916RWbXuW0wd2.jpg)

## Supported Types 🌈

- int64 


- float64

- bool

- objects


- datetime64/timedelta


- [Category](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html)

- The category type is good when the same elements occur over and over again.
- If the number of categories approaches the length of the data, it will more memory consuming than an equivalent object dtype representation
- New in version 0.23.0.

- [Sparse Types](https://pandas.pydata.org/docs/user_guide/sparse.html)


- The sparse types is good when most of the array includes nulls.
- New in version 0.24.0

- [Nullable Integer](https://pandas.pydata.org/docs/user_guide/integer_na.html)/[Nullable Bolean](https://pandas.pydata.org/docs/user_guide/boolean.html)


- The Nullable type is good when element are integer/boolean and includes nulls.
- This is because NaN is a float and it forces the entire array to be cast as float and thus has a bigger memory footprint. 
- New in version 0.24.0

so its pretty obvious that we should aim for the type that has lowest memory footprint with the same functionality

## Optimizing Types 🌈

- Loading dataframes with specific types

- Use to_numeric/to_datetime/to_timedelta functions with downcast parameter

For example it will try to cast float into int if the array only contain regular numbers

In [ ]:
# Naive yet powerfull optimization method.
# do not use in production

def optimize_types(df):    
    # Optimize dates 
    dates = [column for column in df.columns if is_datetime(df[column])]
    df[dates] = df[dates].apply(pd.to_datetime, downcast='integer')
    
    # Optimize strings
    categorical = [col for col in df.select_dtypes(include=['object']).columns.tolist()
                   if col not in dates and float(len(df[col].unique())) / len(df[col]) < 0.5]
    df[categorical] = df[categorical].astype('category')
 
    # Optimize ints
    ints = df.select_dtypes(include=['int64']).columns.tolist()
    df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')
    
    # Optimize floats     
    floats = df.select_dtypes(include=['float64']).columns.tolist()
    df[floats] = df[floats].apply(pd.to_numeric, downcast='float')
    
    return df

In [12]:
df = load_dataset(naivly=False)
df = secret_weapon.optimize_types(df)

## Optimized Types 🌈

### Dataframe Size 🏋️


In [13]:
df.memory_usage(deep=True).sum()


36999962

### Columns Sizes 🏋️

In [14]:
df.memory_usage(deep=True)

Index                8002720
order_id             8963321
date                 2204942
date_of_meal         3942538
participants         5883450
meal_price           2000680
type_of_meal         1000611
heroes_adjustment    1000340
meal_tip             4001360
dtype: int64

In [15]:
df.dtypes

order_id             category
date                 category
date_of_meal         category
participants         category
meal_price              int16
type_of_meal         category
heroes_adjustment        bool
meal_tip              float32
dtype: object

## 12x Improvement In Memory ⌛

![](https://vignette.wikia.nocookie.net/memepediadankmemes/images/8/80/Acb.jpg/revision/latest/scale-to-width-down/340?cb=20180822064733)

## Types Magic Explanation 🧙


- DataFrame is stored in seperated blocks depending on thier types.


- Under the hood, pandas groups the columns into blocks of values of the same type.


- For blocks representing numeric values like integers and floats, pandas combines the columns and stores them as a NumPy ndarray. Also for extension arrays.

- The NumPy ndarray is built around a C array, and the values are stored in a contiguous block of memory. Due to this storage scheme, accessing a slice of values is incredibly fast.

![](https://www.dataquest.io/wp-content/uploads/2019/01/df_blocks.png)

## Custom Types 🦸🏼🦸‍♀️

- This optimization is for extreme cases as it requires a lot of effort and skills.
- basicly its implementing you own custom types using extension array

- [Your Own Types](https://www.youtube.com/watch?v=xx7H5EkzQH0)

- Open Sourced Types like [cyberpandas](https://github.com/ContinuumIO/cyberpandas) and [geopandas](https://github.com/geopandas/geopandas) 

There are open sourced Types like cyberpandas for ip like objects and geopadnas for spatial like objects


![](https://i.kym-cdn.com/photos/images/facebook/000/549/339/210.jpg)

## Pandas Usage 🐼

- Concat vs Append ➕

- GroupBy Optimizations 👩‍👩‍👧

- Merge Optimization 🔍

- numeric_only parameter 🧮

- Query/Eval 🧬

## Concat vs Append ➕


In [17]:
%%timeit
slow_df = pd.DataFrame({})
for i in range(10):
    slow_df.append(pd.read_csv(path))
slow_df

KeyboardInterrupt: 

In [18]:
%%timeit
fast_df = pd.concat([pd.read_csv(path) for i in range(10)])

KeyboardInterrupt: 

- Since append creates a new object with the combined data.

- it inefficient when doing multiple append operations

- Thus it is generally better to build a list of DataFrames and using concat.

## % 30 Performance Improvement⌛

## GroupBy Optimizations 👩‍👩‍👧

- When possible filter first

- Avoid apply/transform [when possible](https://medium.com/dunder-data/pandas-challenge-001-optimize-custom-grouping-function-44a9f2f321f8)

## Merge Optimization 🔍

- [Join on index](https://stackoverflow.com/questions/53645882/pandas-merging-101)

- When possible filter first

- When possible aggregate first

## numeric_only parameter 🧮


In [ ]:
%%timeit
df[["meal_price", "meal_tip"]].mean()

In [ ]:
%%timeit
df[["meal_price", "meal_tip"]]mean(numeric_only=True)

- There are many methods including Mean/Sum/Mode/Min/ and max that has this parameter
- Work on dataframes and not series

## % 30 Performance Improvement⌛

## Query/Eval 🧬

- Improve Execution Time

- To benefit from using eval and query it uses numexpr if installed.

- The pandas API indicate to only use eval on  more than 10,000 rows, as the traditional method is faster for smaller arrays.

- Just a reminder use this optimization only if its your bottle neck

- The expected improvment can be up to X2

- Improve Memory

- The main benefit of eval/query is the saved memory

- NumPy allocate memory to every itermediate step.

- The Numexpr library gives you the ability to compute this type of compound expression element by element, without the need to allocate full intermediate arrays.

- Not all Operations are supported

## Example

In [18]:
%%timeit
df[df.type_of_meal=="Breakfast"]

18.8 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
df.query("type_of_meal=='Breakfast'")

25.9 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## % 30 Performance Improvement⌛

## Additional Resources 📚

- [Pandas Index Explained](https://towardsdatascience.com/pandas-index-explained-b131beaf6f7b)


- [Merge Optimization](https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2)

- [Eval & Query](https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html)

![](https://i.imgflip.com/1px0z1.jpg)

## Compiled Code 🤯

- Pure Python's code does some operations very slowly. 
- Due to its dynamic nature.
- sequences of operations cannot be compiled down to efficient machine code as in other languages like C and Fortran. 
- There have been various attempts to address this weakness

- Cython

- the Cython project, which converts Python code to compilable C code; 

- Numba

- and the second one the Numba project, which converts snippets of Python code to fast LLVM bytecode

## Cython 🤯

- Up to 50x speedup from pure python 👍


- Learning Curve 👎

- Separated Compilation Step 👎 👍


- The compilation is both a con and a pro
- on one part, there is an additional work to make the compiled code work cleanly with our code
- on the other part, there is no compilation overhead on runtime

## Example 

In [ ]:
def foo(N):
    accumulator = 0
    for i in range(N):
        accumulator = accumulator + i
    avg = accumulator / N
    return avg

In [ ]:
%%timeit
df.meal_price.map(foo)

In [22]:
%load_ext Cython

In [23]:
%%cython
def cython_foo(long N):
    cdef long adding_total
    adding_total = 0

    cdef long i
    for i in range(N):
        adding_total += i

    return adding_total

In [24]:
%%timeit
df.meal_price.map(cython_foo)

349 ms ± 3.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 48x  Performance Improvement⌛

## Numba 🤯

- Up to 200x speedup from pure python  👍


- Easy 👍


using numba is really easy its simply adding a decorator to a method

- Highly Configurable  👍

- Debugging 👎 👍

- Debugging is both a con and a pro
- on one part, debugging the buissness logic is easy we can just remove the decorator
- on the other part, if there is an issue in the compiled code itself its not quite easy to debug these issues.

- Mostly Numeric 👎 

## Example 

In [25]:
@jit(nopython=True)
def numba_foo(N):
    accumulator = 0
    for i in range(N):
        accumulator = accumulator + i
    avg = accumulator / N
    return avg

In [26]:
%%timeit
df.meal_price.map(numba_foo)

274 ms ± 3.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 63x  Performance Improvement⌛

## Additional Resources 📚

- [Numba Workshop](https://www.youtube.com/watch?v=1AwG0T4gaO0) 🛠

- [Cython Workshop](https://www.youtube.com/watch?v=FepqwPI6U80)🛠


- [Numba VS C++](https://murillogroupmsu.com/numba-versus-c/) ⚖

- [Cython VS C](https://notes-on-cython.readthedocs.io/en/latest/std_dev.html) ⚖

## General Pyhon Optimization 🐍

- Since I want to focus on pandas related optimization 
- i will go brifly on the listed optimizations
- The first one is caching 

- [Caching](https://medium.com/datadriveninvestor/all-things-caching-use-cases-benefits-strategies-choosing-a-caching-technology-exploring-fa6c1f2e93aa) 🏎


- By using a cached function result whenever it is safe
- will make our code avoid unnecessary work.

- [Intermediate Variables](https://pythonspeed.com/articles/function-calls-prevent-garbage-collection/) 👩‍👩‍👧‍👧

- In many cases we will do intermidiate calculation in the same method
- For example first we will calculate the relevent rows and save it in a variable 
- and then calculate the sum.
- thus untill garbage collection take place we have memory foot print of an object we dont longer require

-  [Concurrency And Parallelism](https://medium.com/fintechexplained/advanced-python-concurrency-and-parallelism-82e378f26ced) 🎸🎺🎻🎷

- native pandas work on a single process and we might want to gain an optimization boost by adding parallelism and concurrency
- There are many more general python optimizations 
- and the high performance python book is a great read

- [High Performance Python Book ](https://www.amazon.com/High-Performance-Python-Performant-Programming/dp/1449361595) 📖

## Others Optimizations✳️ 

- [Serialization Time](https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d)


we can improve the loading time of our dataset by choosing to persist the data in a diffrent format, whether parquet arrow or another there is a great comparision attached.

- [Chunks](https://pandas.pydata.org/docs/user_guide/scale.html)

- Some workloads can be achieved with chunking.
- chunking is splitting a large problem into a bunch of small problems.
- As long as each chunk fits in memory, you can work with datasets that are much larger than memory.
- Chunking works well when the operation you’re performing requires zero or minimal coordination between chunks.

- [Pandas Alternatives](https://www.youtube.com/watch?v=RRtqIagk93k)
    - [Cudf](https://www.youtube.com/watch?v=lV7rtDW94do)
    - [pyspark](http://spark.apache.org/docs/latest/api/python/index.html)
    - [modin](https://www.youtube.com/watch?v=-HjLd_3ahCw)

- For more complicated workflows, you’re better off using another library.
- There are many libraries that provide dataframe api.
- some are parrallal some use gpu 
- but i wont into it

![](https://i.pinimg.com/originals/b9/0a/79/b90a79b4c361d079144597d0bcdd61de.jpg)